In [2]:
import transformers
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from datasets import load_dataset, load_metric
import pandas as pd
from datasets import ClassLabel, Sequence
from transformers import AutoTokenizer
from transformers import EarlyStoppingCallback
from transformers import default_data_collator
import collections
import numpy as np
from datasets import Dataset

In [21]:
squad_v2 = False
model_checkpoint = "bert-base-uncased"
batch_size = 16
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

pad_on_right = tokenizer.padding_side == "right"
max_length = 384 # The maximum length of a feature (question and context)Q
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.


model = AutoModelForQuestionAnswering.from_pretrained('./saved_model')
data_collator = default_data_collator

num_samples = 200
sample_size = 50

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/luki/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/

In [5]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad_with_callbacks",
    evaluation_strategy = "steps",
    eval_steps = 500, # Evaluation and Save happens every 10 steps
    save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none",
    load_best_model_at_end=True,
)

In [6]:
trainer = Trainer(
    model,
    args,
    data_collator=data_collator,
    tokenizer=tokenizer,    
    callbacks = [EarlyStoppingCallback(early_stopping_patience=8)]
)

In [8]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [9]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue
                    if len(offset_mapping[start_index]) == 0 or len(offset_mapping[end_index]) == 0: 
                        continue
                                        
                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [10]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

In [24]:
def compute_metrics_for_sample(sample):
    validation_features = sample.map(
        prepare_validation_features,
        batched=True,
        remove_columns=sample.column_names
    )

    raw_predictions = trainer.predict(validation_features)

    validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

    final_predictions = postprocess_qa_predictions(sample, validation_features, raw_predictions.predictions)

    if squad_v2:
        formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
    else:
        formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in sample]
    return metric.compute(predictions=formatted_predictions, references=references)

In [25]:
data_lower = pd.read_json('valid_data_lower_distance_than_4.json')

exact_list_lower = []
f1_list_lower = []

for i in range(num_samples):
    df = data_lower.sample(n=sample_size)
    sample = Dataset.from_pandas(df)
    metrics1 = compute_metrics_for_sample(sample)
    exact_list_lower.append(metrics1['exact_match'])
    f1_list_lower.append(metrics1['f1'])


  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 51
  Batch size = 16


Post-processing 50 example predictions split into 51 features.


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 51
  Batch size = 16


Post-processing 50 example predictions split into 51 features.


  0%|          | 0/50 [00:00<?, ?it/s]

[80.0, 80.0, 90.0, 78.0, 84.0, 78.0, 92.0, 92.0, 92.0, 82.0, 86.0, 80.0, 88.0, 84.0, 84.0, 88.0, 90.0, 84.0, 76.0, 84.0, 78.0, 86.0, 88.0, 86.0, 88.0, 88.0, 78.0, 88.0, 80.0, 80.0, 90.0, 82.0, 82.0, 84.0, 92.0, 88.0, 84.0, 90.0, 86.0, 82.0, 80.0, 78.0, 84.0, 80.0, 86.0, 86.0, 88.0, 88.0, 84.0, 86.0, 90.0, 78.0, 78.0, 88.0, 80.0, 92.0, 82.0, 88.0, 80.0, 82.0, 90.0, 82.0, 76.0, 78.0, 84.0, 82.0, 90.0, 82.0, 76.0, 80.0, 86.0, 84.0, 88.0, 88.0, 78.0, 82.0, 78.0, 86.0, 78.0, 74.0, 88.0, 82.0, 82.0, 88.0, 82.0, 92.0, 82.0, 92.0, 86.0, 82.0, 84.0, 92.0, 96.0, 84.0, 80.0, 82.0, 82.0, 80.0, 88.0, 88.0, 84.0, 80.0, 86.0, 78.0, 88.0, 88.0, 92.0, 82.0, 82.0, 88.0, 88.0, 84.0, 90.0, 84.0, 84.0, 76.0, 78.0, 84.0, 84.0, 80.0, 88.0, 88.0, 82.0, 86.0, 80.0, 78.0, 78.0, 88.0, 92.0, 86.0, 82.0, 82.0, 84.0, 78.0, 82.0, 80.0, 84.0, 84.0, 90.0, 82.0, 76.0, 84.0, 88.0, 94.0, 86.0, 78.0, 84.0, 88.0, 78.0, 84.0, 72.0, 92.0, 78.0, 72.0, 76.0, 84.0, 78.0, 90.0, 80.0, 84.0, 82.0, 88.0, 86.0, 84.0, 84.0, 86.0, 88.

In [28]:
d_lower = {'exact_match': exact_list_lower, 'f1': f1_list_lower}
d_lower
lower_than_4 = pd.DataFrame(d_lower)
lower_than_4

,exact_match,f1
0,80.0,89.267495
1,80.0,88.570563
2,90.0,95.168831
3,78.0,90.776636
4,84.0,91.521303
...,...,...
195,78.0,87.004113
196,82.0,89.190476
197,88.0,93.866667
198,84.0,91.580513


In [29]:
lower_than_4.describe()

,exact_match,f1
count,200.000000,200.000000
mean,83.870000,90.768952
std,4.812369,3.394058
min,72.000000,81.233650
25%,80.000000,88.159314
50%,84.000000,90.820564
75%,88.000000,93.431095
max,96.000000,98.376912


In [30]:
data_higher = pd.read_json('valid_data_higher_distance_than_4.json')

exact_list_higher = []
f1_list_higher = []

for i in range(num_samples):
    df = data_higher.sample(n=sample_size)
    sample = Dataset.from_pandas(df)
    metrics1 = compute_metrics_for_sample(sample)
    exact_list_higher.append(metrics1['exact_match'])
    f1_list_higher.append(metrics1['f1'])

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 51
  Batch size = 16


Post-processing 50 example predictions split into 51 features.


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 52
  Batch size = 16


Post-processing 50 example predictions split into 52 features.


  0%|          | 0/50 [00:00<?, ?it/s]

In [31]:
d_higher = {'exact_match': exact_list_higher, 'f1': f1_list_higher}
d_higher
higher_than_4 = pd.DataFrame(d_higher)
higher_than_4

,exact_match,f1
0,84.0,88.948718
1,68.0,80.530952
2,76.0,85.393651
3,80.0,84.555556
4,82.0,87.674747
...,...,...
195,70.0,79.440206
196,64.0,71.743590
197,74.0,77.857879
198,76.0,86.188456


In [32]:
higher_than_4.describe()

,exact_match,f1
count,200.000000,200.000000
mean,75.680000,84.494283
std,5.816075,4.174477
min,62.000000,71.743590
25%,72.000000,81.556891
50%,76.000000,84.625146
75%,80.000000,87.650183
max,92.000000,96.131868


In [39]:
lower_than_4.quantile([0.025, 0.975])

,exact_match,f1
0.025,75.95,84.336014
0.975,92.00,97.007498


In [40]:
higher_than_4.quantile([0.025, 0.975])

,exact_match,f1
0.025,64.0,77.455956
0.975,88.0,92.108751


In [36]:
lower_than_4.describe()

,exact_match,f1
count,200.000000,200.000000
mean,83.870000,90.768952
std,4.812369,3.394058
min,72.000000,81.233650
25%,80.000000,88.159314
50%,84.000000,90.820564
75%,88.000000,93.431095
max,96.000000,98.376912


In [37]:
higher_than_4.describe()

,exact_match,f1
count,200.000000,200.000000
mean,75.680000,84.494283
std,5.816075,4.174477
min,62.000000,71.743590
25%,72.000000,81.556891
50%,76.000000,84.625146
75%,80.000000,87.650183
max,92.000000,96.131868
